In [ ]:
!pip install melee

In [ ]:
agentState = [
    0, # agent character
    [0,0], # x and y position of current position (float)
    [0,0,0], # user ground x, air x and y speed of character (float)
    [0,0], # attack-based x and y speed of character (float)
    True, # direction character is facing (left is false, right is true)
    False, # is character offstage
    2, # number of jumps left for character (int)
    0, # what percent is the character at (int)
    0, # number of stocks remaining for character (int)
    0, # number of hitstun frames left for character (0 for none) (int)
    0, # number of invulnerability frames left for character (0 for none) (int)
    60, # amount of shield health enemy character has (float)
    0, # current action being taken by character (0 if none... maybe)
    0, # current frame of action character is in (starts at 1 so if 0 no action is being taken)
    0, # distance from enemy character (float)
    0, # opponents character
    [0,0], # x and y position of enemy character (float)
    [0,0,0], # user ground x, air x and y speed of enemy character (float)
    [0,0], # attack-based x and y speed of enemy character (float)
    True, # direction enemy character is facing (left is false, right is true)
    False, # is enemy character offstage
    2, # number of jumps left for enemy character (int)
    0, # what percent is the enemy character at (int)
    0, # number of stocks remaining for enemy character (int)
    0, # number of hitstun frames left for enemy character (0 for none) (int)
    0, # number of invulnerability frames left for enemy character (0 for none) (int)
    60, # amount of shield health enemy character has (float)
    0, # current action being taken by enemy character (0 if none... maybe)
    0, # current frame of action enemy character is in (starts at 1 so if 0 no action is being taken)
    0 #stage currently being played on (for this project we will only be training on one stage) 
]
# expected parameters: gamestate variable from Dolphin, port of agent, opponent's port, agentState variable we are writing to
# returns agentState
def gamestateToAgentstate(gamestate, character_port, enemy_port, agentState):
    agentState[0] = gamestate.players[character_port].character
    agentState[1] = [gamestate.players[character_port].position.x , gamestate.players[character_port].position.y]
    agentState[2] = [gamestate.players[character_port].speed_ground_x_self , gamestate.players[character_port].speed_air_x_self, gamestate.players[character_port].speed_y_self]
    agentState[3] = [gamestate.players[character_port].speed_x_attack , gamestate.players[character_port].speed_y_attack]
    agentState[4] = gamestate.players[character_port].facing
    agentState[5] = gamestate.players[character_port].off_stage
    agentState[6] = gamestate.players[character_port].jumps_left
    agentState[7] = gamestate.players[character_port].percent
    agentState[8] = gamestate.players[character_port].stock
    agentState[9] = gamestate.players[character_port].hitstun_frames_left
    agentState[10] = gamestate.players[character_port].invulnerability_left
    agentState[11] = gamestate.players[character_port].shield_strength
    agentState[12] = gamestate.players[character_port].action
    agentState[13] = gamestate.players[character_port].action_frame
    agentState[14] = gamestate.distance
    agentState[15] = gamestate.players[enemy_port].character
    agentState[16] = [gamestate.players[enemy_port].position.x , gamestate.players[enemy_port].position.y]
    agentState[17] = [gamestate.players[enemy_port].speed_ground_x_self , gamestate.players[enemy_port].speed_air_x_self, gamestate.players[enemy_port].speed_y_self]
    agentState[18] = [gamestate.players[enemy_port].speed_x_attack , gamestate.players[enemy_port].speed_y_attack]
    agentState[19] = gamestate.players[enemy_port].facing
    agentState[20] = gamestate.players[enemy_port].off_stage
    agentState[21] = gamestate.players[enemy_port].jumps_left
    agentState[22] = gamestate.players[enemy_port].percent
    agentState[23] = gamestate.players[enemy_port].stock
    agentState[24] = gamestate.players[enemy_port].hitstun_frames_left
    agentState[25] = gamestate.players[enemy_port].invulnerability_left
    agentState[26] = gamestate.players[enemy_port].shield_strength
    agentState[27] = gamestate.players[enemy_port].action
    agentState[28] = gamestate.players[enemy_port].action_frame
    agentState[29] = gamestate.stage

In [ ]:
import argparse
import signal
import sys
import melee
import random

console = melee.Console(path=r"C:\Users\carte\Desktop\Games\Melee Emulator")

controller = melee.Controller(console=console, port=2)
controller_CPU = melee.Controller(console=console, port=1)
#controller_human = melee.Controller(console=console,port=1,type=melee.ControllerType.GCN_ADAPTER)

console.run()
console.connect()

controller.connect()
controller_CPU.connect()
#controller_human.connect()
costume = 3
temp_a = 0

def signal_handler(sig, frame):
    console.stop()
    if args.debug:
        log.writelog()
        print("") #because the ^C will be on the terminal
        print("Log file created: " + log.filename)
    print("Shutting down cleanly...")
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

while True:
    gamestate = console.step()
    
    if gamestate is None:
        continue
        
    if console.processingtime * 1000 > 12:
        print("WARNING: Last frame took " + str(console.processingtime*1000) + "ms to process.")

    if gamestate.menu_state in [melee.Menu.IN_GAME, melee.Menu.SUDDEN_DEATH]:
        gamestateToAgentstate(gamestate, 2, 1, agentState)
        #print(agentState)
        if gamestate.distance < 20:
            controller.press_button(melee.enums.Button.BUTTON_B)
            controller.tilt_analog(melee.enums.Button.BUTTON_MAIN, 0.5, 0)
        else:
            onleft = gamestate.players[2].position.x < gamestate.players[1].position.x
            controller.tilt_analog(melee.enums.Button.BUTTON_MAIN, int(onleft), 0.5)
            controller.release_button(melee.enums.Button.BUTTON_B)
            
            if gamestate.players[2].position.y < gamestate.players[1].position.y:
                controller.press_button(melee.enums.Button.BUTTON_X)
            else:
                controller.release_button(melee.enums.Button.BUTTON_X)
             
            
    else:
        melee.menuhelper.MenuHelper.menu_helper_simple(gamestate=gamestate,controller=controller,character_selected=melee.enums.Character.CPTFALCON,stage_selected=melee.enums.Stage.YOSHIS_STORY,autostart=False)
        if gamestate.menu_state == melee.enums.Menu.CHARACTER_SELECT:
            if gamestate.players[1].cpu_level != 9 or gamestate.ready_to_start >= 7:
                melee.menuhelper.MenuHelper.choose_character(gamestate=gamestate,controller=controller_CPU,character=melee.enums.Character.LUIGI,cpu_level=9)
            else:
                controller_CPU.empty_input()
                melee.menuhelper.MenuHelper.menu_helper_simple(gamestate=gamestate,controller=controller,character_selected=melee.enums.Character.CPTFALCON,stage_selected=melee.enums.Stage.YOSHIS_STORY,autostart=True)
        else:
            controller_CPU.empty_input()